# Instructions
Just run everything


# 1 - Data Preprocessing (Personality chat datasets)

## 1.1. Download Dataset (Personality chat datasets)

In [0]:
import json
import re

import tensorflow as tf
import numpy as np
import pandas as pd
import nltk

nltk.download('punkt')


# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
    100% |████████████████████████████████| 993kB 20.1MB/s 


In [0]:
#qna_chitchat_the_professional
id = '17BDhIH-aykl7F1dnzXLkn7E77L7yNdnt' #checked
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('qna_chitchat_the_professional.tsv')  

#
id = '1IGyUQuEklPFu0pqlqUY-R_jp1OFXHjdK' #checked
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('qna_chitchat_the_friend.tsv')  

#
id = '1yYl2Zdp1dgP2h2l2GilM3MKXUnY_GaLi' #checked
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('qna_chitchat_the_comic.tsv')  


## 1.2. Preprocess data (Personality chat datasets)

*You are required to describe which data preprocessing techniques were conducted with justification of your decision. *


An typical approach for normalising words  is to decapitalising words(to lower case), removing non-alphabet, removing stop-word, stemming, lemmatization etc.

I have tried all of above and I found the most effectives are decapitalisation(to lower case) and removing non-alphabet(e.g. puntuation, numebrs...) so I only used these 2. 

Lowering case of the words does not chage the meaning of a given word, and it is not costly to create a extra vocab for 2 words with exactly same meaning. Removing non-alphabet also an effective step since t they themself are not word and has no meaning at all.

Stop-word removal may actually harm the model as the questions set contains too  many stopwords such as what, when, how... are, do, our, you... by removing stopwords the whole question set contains only single or even empty tokens list of a sentence. And i did tried apply removing stops words and it actually increase difficulty to converge the models(training loss stops reducing in an very early iteration no matter what setting of hyper parameters such as leanring rate).

stemming, lemmatization do not help much for training the model or chatbot evaluation(observe if chatting content making sense)





In [0]:
from nltk.tokenize import word_tokenize, sent_tokenize

def get_unique_word(file_name): #return max_input_words_amount, unique_words; num_dic dic_len can be inferenced from the unique_word
    # Sequence data
    # Generate unique tokens list from qas.json
    seq_data = []
    whole_words = []
    max_input_words_amount = 0
    max_output_words_amount = 1

    from nltk import word_tokenize
    df = pd.read_csv(file_name, sep="\t")
    for index, row in df.iterrows():

        question = row['Question']
        answer = row['Answer']

        seq_data.append([question, answer]) #TODO: remove spacing
      
        # we need to tokenise question    
        tokenized_q = word_tokenize(question)
        #print('before', tokenized_q)
        tokenized_q = [token for token in tokenized_q if token.isalpha()] #come back later
        tokenized_q = [token.lower() for token in tokenized_q]
        #print('after', tokenized_q)

        # we do not need to tokenise answer (because we implement N to One model)
        # make a list with only one element (whole sentence)
        tokenized_a = [answer]


        # add question list and answer list (one element)
        whole_words += tokenized_q
        whole_words += tokenized_a

        # we need to decide the maximum size of input word tokens
        max_input_words_amount = max(len(tokenized_q), max_input_words_amount) #TODO: ????


    # we now have a vacabulary list
    unique_words = list(set(whole_words))

    # adding special tokens in the vocabulary list    
    # _B_: Beginning of Sequence
    # _E_: Ending of Sequence
    # _P_: Padding of Sequence - for different size input
    # _U_: Unknown element of Sequence - for different size input

    unique_words.append('_B_')
    unique_words.append('_E_')
    unique_words.append('_P_')
    unique_words.append('_U_')


    #num_dic = {n: i for i, n in enumerate(unique_words)}
    #dic_len = len(num_dic)

    return (seq_data, max_input_words_amount, unique_words)


####Professional unique_words

In [0]:
professional_seq_data, professional_max_input_words_amount, professional_unique_words = get_unique_word('qna_chitchat_the_professional.tsv')

In [0]:
professional_unique_words.sort() #sort the unique words

In [0]:
professional_unique_words[:10]

["Age doesn't really apply to me.",
 'Certainly.',
 'Excellent.',
 "Glad you're pleased!",
 'Good evening.',
 'Good morning.',
 'Good night.',
 'Good, thanks.',
 'Goodbye.',
 'Got it.']

In [0]:
professional_num_dic = {n: i for i, n in enumerate(professional_unique_words)}
professional_dic_len = len(professional_num_dic)

In [0]:
print(professional_seq_data) #question: answer pairs
print(professional_max_input_words_amount)
print(professional_unique_words)

[["What's your age?", "Age doesn't really apply to me."], ['Are you young?', "Age doesn't really apply to me."], ['When were you born?', "Age doesn't really apply to me."], ['What age are you?', "Age doesn't really apply to me."], ['Are you old?', "Age doesn't really apply to me."], ['How old are you?', "Age doesn't really apply to me."], ['How long ago were you born?', "Age doesn't really apply to me."], ['Ask me anything', "I'm better at answering questions."], ['Ask me a question', "I'm better at answering questions."], ['Can you ask me a question?', "I'm better at answering questions."], ['Ask me something', "I'm better at answering questions."], ['What do you want to know about me?', "I'm better at answering questions."], ['Can you sleep?', "I don't have a body."], ['Do you have boogers?', "I don't have a body."], ["Don't you ever sleep?", "I don't have a body."], ['Do you dream?', "I don't have a body."], ['Do you smell?', "I don't have a body."], ['Do you sweat?', "I don't have 

In [0]:
print(professional_num_dic)
print(professional_dic_len)

{'I aim to serve.': 0, 'rofl': 1, 'marry': 2, 'tomorrow': 3, "I try, but I don't always get it right.": 4, 'rest': 5, 'available': 6, 'meet': 7, 'mood': 8, 'Good night.': 9, 'question': 10, 'correct': 11, 'even': 12, 'bisexual': 13, 'bump': 14, 'purpose': 15, 'sadness': 16, 'work': 17, 'hello': 18, 'working': 19, 'mad': 20, 'life': 21, 'should': 22, 'try': 23, 'look': 24, 'gender': 25, 'answer': 26, 's': 27, 'later': 28, 'hate': 29, 'hear': 30, 'awesome': 31, "I'm digital. In other words, I'm not human.": 32, 'bye': 33, 'why': 34, 'Moving on.': 35, 'famished': 36, 'the': 37, 'better': 38, "That's not me, but hello.": 39, "Sorry. That's not something I can do.": 40, 'kinds': 41, 'interest': 42, 'in': 43, "I think it's best if we stick to a professional relationship.": 44, 'makes': 45, 'let': 46, 'af': 47, 'song': 48, 'garden': 49, 'here': 50, 'bingo': 51, 'stop': 52, 'smell': 53, "Well, let me know if there's anything I can do for you.": 54, 'come': 55, 'thing': 56, 'Excellent.': 57, 'r

####Friend unique_word

In [0]:
friend_seq_data, friend_max_input_words_amount, friend_unique_words = get_unique_word('qna_chitchat_the_friend.tsv')

In [0]:
friend_num_dic = {n: i for i, n in enumerate(friend_unique_words)}
friend_dic_len = len(friend_num_dic)

In [0]:
friend_unique_words.sort() #sort

In [0]:
friend_unique_words[:10]

['And to you as well!',
 'Aw nuts.',
 "Aw, I'm blushing.",
 'Awesome.',
 'BFFs!',
 'Bye.',
 'Chat away!',
 'Cool!',
 'Cool.',
 "Definitely didn't see that coming!"]

In [0]:
print(friend_seq_data) #question: answer pairs
print(friend_max_input_words_amount)
print(friend_unique_words)

[["What's your age?", "I don't really have an age."], ['Are you young?', "I don't really have an age."], ['When were you born?', "I don't really have an age."], ['What age are you?', "I don't really have an age."], ['Are you old?', "I don't really have an age."], ['How old are you?', "I don't really have an age."], ['How long ago were you born?', "I don't really have an age."], ['Ask me anything', "I'm a much better answerer than asker."], ['Ask me a question', "I'm a much better answerer than asker."], ['Can you ask me a question?', "I'm a much better answerer than asker."], ['Ask me something', "I'm a much better answerer than asker."], ['What do you want to know about me?', "I'm a much better answerer than asker."], ['Can you sleep?', "I don't have the hardware for that."], ['Do you have boogers?', "I don't have the hardware for that."], ["Don't you ever sleep?", "I don't have the hardware for that."], ['Do you dream?', "I don't have the hardware for that."], ['Do you smell?', "I do

In [0]:
print(friend_num_dic)
print(friend_dic_len)

{'everything': 0, 'sounds': 1, 'correct': 2, 'tune': 3, "That doesn't really apply to me.": 4, 'drive': 5, 'terrible': 6, "Why do seagulls fly over the sea? Because if they flew over the bay, they'd be bagels.": 7, 'i': 8, 'cook': 9, 'pansexual': 10, 'blue': 11, 'Chat away!': 12, 'morning': 13, 'question': 14, 'fun': 15, 'prefer': 16, 'robot': 17, 'tuckered': 18, 'other': 19, 'alexa': 20, 'And to you as well!': 21, 'So happy!': 22, 'partner': 23, 'ago': 24, 'missed': 25, 'Talk to you later!': 26, 'prettier': 27, 'bad': 28, 'know': 29, 'kind': 30, "Sorry, I can't really speak to that.": 31, 'tired': 32, 'real': 33, 'hang': 34, "I'm a much better answerer than asker.": 35, 'zip': 36, 'wish': 37, 'tell': 38, "Thanks! You're pretty cool yourself.": 39, 'ticked': 40, 'you': 41, 'time': 42, 'hello': 43, 'same': 44, "I've heard really good things about naps.": 45, 'husband': 46, 'sandwich': 47, 'sec': 48, 'technology': 49, 'be': 50, 'at': 51, 'appreciate': 52, 'your': 53, 'play': 54, 'red': 5

####Comic unique_word

In [0]:
comic_seq_data, comic_max_input_words_amount, comic_unique_words = get_unique_word('qna_chitchat_the_comic.tsv')

In [0]:
comic_unique_words.sort()

In [0]:
comic_unique_words[:10]

['*Fist bump*',
 'Agreed. It is awesome to meet me.',
 'All those years at charm school. Wasted.',
 'Alright, cool.',
 'Avoiding the subject in 3, 2, 1... Hi there!',
 'Back atcha.',
 'Behold the field in which I grow my jokes and see that it is barren.',
 "Can't complain. I literally can't complain.",
 'Cartoonish supervillainy is beneath me. And beyond me.',
 'Cool.']

In [0]:
comic_num_dic = {n: i for i, n in enumerate(comic_unique_words)}
comic_dic_len = len(comic_num_dic)

In [0]:
print(comic_seq_data) #question: answer pairs
print(comic_max_input_words_amount)
print(comic_unique_words)

[["What's your age?", "I'm age-free."], ['Are you young?', "I'm age-free."], ['When were you born?', "I'm age-free."], ['What age are you?', "I'm age-free."], ['Are you old?', "I'm age-free."], ['How old are you?', "I'm age-free."], ['How long ago were you born?', "I'm age-free."], ['Ask me anything', "Nah, I'm good."], ['Ask me a question', "Nah, I'm good."], ['Can you ask me a question?', "Nah, I'm good."], ['Ask me something', "Nah, I'm good."], ['What do you want to know about me?', "Nah, I'm good."], ['Can you sleep?', 'Not so far.'], ['Do you have boogers?', 'Not so far.'], ["Don't you ever sleep?", 'Not so far.'], ['Do you dream?', 'Not so far.'], ['Do you smell?', 'Not so far.'], ['Do you sweat?', 'Not so far.'], ['Do you get tired?', 'Not so far.'], ['Can you sneeze?', 'Not so far.'], ['Getting tired of you', 'Sometimes I like to take a break from being awesome.'], ['You bore me', 'Sometimes I like to take a break from being awesome.'], ["I'm tired of you", 'Sometimes I like t

In [0]:
print(comic_num_dic)
print(comic_dic_len)

{'rofl': 0, "You're super okay.": 1, 'marry': 2, 'tomorrow': 3, 'rest': 4, 'How many humans does it take to screw in a light bulb? One.': 5, 'available': 6, 'meet': 7, 'mood': 8, 'question': 9, 'correct': 10, 'even': 11, 'bisexual': 12, 'bump': 13, 'purpose': 14, 'sadness': 15, 'work': 16, 'hello': 17, 'working': 18, 'mad': 19, 'life': 20, 'should': 21, "I'm enough for me.": 22, 'try': 23, 'look': 24, 'gender': 25, 'answer': 26, 's': 27, 'later': 28, 'hate': 29, "I still haven't found a moisturizer that works for me.": 30, 'hear': 31, 'Cool.': 32, 'awesome': 33, 'bye': 34, 'why': 35, 'Moving on.': 36, 'famished': 37, 'Alright, cool.': 38, 'the': 39, 'better': 40, 'kinds': 41, 'Oh.': 42, 'interest': 43, 'in': 44, 'makes': 45, 'let': 46, 'af': 47, 'song': 48, 'garden': 49, 'here': 50, 'bingo': 51, 'stop': 52, 'smell': 53, 'come': 54, 'thing': 55, 'I just have one answer for each kind of question. Try asking about something new.': 56, 'red': 57, 'boring': 58, 'sister': 59, 'boss': 60, 'se

# 2 - Model Implementation

## 2.1. Word Embeddings

In [0]:
# Please comment your code

In [0]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize

from gensim.models import Word2Vec




### 2.1.1. Download Dataset for Word Embeddings



*You are required to describe which data was used with justification of your decision.*

I used all 3 Microsoft chatbot dataset(pro, friend and comic) to train the model.
I basically create a lists of sentense tokens for all 3 data set  and the data proprocessing steps I included are:


*   remove puntuation, numbers and any non-alphabet
*   keep lower case only


This is basically an typical example of normalising words. The main reason is to is to treat all words on an equal footing and allow for uniform processing. For instance, lower or upper case doesn't chage the meaning of a given word, it is not worth to create a extra vocab for 2 word which are exactly the same. Removing puntuationa and another non-alphabet also an essential step s since they can be intrepred as nosie, as they themself are not word and has no meaning at all.




In [0]:
# Please comment your code

### 2.1.2. Data Preprocessing for Word Embeddings

In [0]:
'''
data preprocessing

'''

'\ndata preprocessing\n\n'

In [0]:
def generate_sentences_qna_chitchat():
  # Sequence 
  # Generate unique tokens list from qas.json
  results = []

  #concat files name
  for filename in ['qna_chitchat_the_professional.tsv', 'qna_chitchat_the_friend.tsv','qna_chitchat_the_comic.tsv']:
    df = pd.read_csv(filename, sep="\t")
    for index, row in df.iterrows():
        question = row['Question']
        answer = row['Answer']
        #tokenize
        tokenized_q = word_tokenize(question)
        tokenized_a = word_tokenize(answer)
        
        results.append(tokenized_q)
        results.append(tokenized_a)
        
  return results



In [0]:
sentences = generate_sentences_qna_chitchat()
print(sentences[:3])


#process the tokens
def preprocess_sentences(sentences):
  results = []
  for sen in sentences:
    sen_tmp = [token for token in sen if token.isalpha()]
    sen_tmp = [token.lower() for token in sen_tmp]
    
    results.append(sen_tmp)
    
  return results

preprocessed_sentences = preprocess_sentences(sentences)
print(preprocessed_sentences[:10])
#

[['What', "'s", 'your', 'age', '?'], ['Age', 'does', "n't", 'really', 'apply', 'to', 'me', '.'], ['Are', 'you', 'young', '?']]
[['what', 'your', 'age'], ['age', 'does', 'really', 'apply', 'to', 'me'], ['are', 'you', 'young'], ['age', 'does', 'really', 'apply', 'to', 'me'], ['when', 'were', 'you', 'born'], ['age', 'does', 'really', 'apply', 'to', 'me'], ['what', 'age', 'are', 'you'], ['age', 'does', 'really', 'apply', 'to', 'me'], ['are', 'you', 'old'], ['age', 'does', 'really', 'apply', 'to', 'me']]


### 2.1.3. Build Word Embeddings Model

*You are required to describe how hyperparameters were decided with justification of your decision.*

My initial goal is to train a word embedding model such that the training loss is as small as possible.


word_size(vocab size) is set to 100 since{} 100, 200, 300} are most common proven effective setting for industries and research works. And I found 100 is the best in my cases in terms of converge speed. As shown in **2.1.4 Train Word Embedding Model**, the model arealy acheived 0 training loss on the firdst few iterations.


learning rate have been tested from 0.001-1 and I found the best is 1 since it converge fast and the training loss is stably decresing as shown below.

Unlike other learning rate such as 0.001, 0.01, 0.5 the loss is flutuated  or converge slowly.(e.g. 0.01 even start with >70000 loss and suddently drop to 20000 then increse to 50000...)






####skip-gram

In [0]:
from gensim.models import Word2Vec

In [0]:
word2vec_params = {
   'sg': 1,  #1 : skip-gram
   "size": 100
}


In [0]:
skip_gram_model = Word2Vec(**word2vec_params)
skip_gram_model.build_vocab(preprocessed_sentences)



### 2.1.4. Train Word Embeddings Model

####skip*gram*

In [0]:
losses = []
learning_rate = 1

for i in range(55):
    trained_word_count, raw_word_count = skip_gram_model.train(sentences, compute_loss=True,
                                                     start_alpha=learning_rate,
                                                     end_alpha=learning_rate,
                                                     total_examples=skip_gram_model.corpus_count,
                                                     epochs=1)
    loss = skip_gram_model.get_latest_training_loss()
    losses.append(loss)
    print(i, loss)

0 0.6462986469268799
1 0.0
2 0.03703863546252251
3 0.0
4 0.0
5 0.0
6 0.0
7 0.0
8 5.775108814239502
9 1.3475273847579956
10 0.0033806420397013426
11 1.8385355472564697
12 0.0
13 0.0
14 3.7673838138580322
15 0.0
16 0.0
17 0.4125640094280243
18 0.0
19 0.0
20 0.0026285594794899225
21 0.348150372505188
22 0.0
23 0.0
24 0.0
25 0.0
26 0.0
27 0.17471018433570862
28 0.0
29 0.0
30 0.0
31 0.0
32 0.0
33 0.0
34 0.0
35 0.0
36 0.0
37 1.7805737257003784
38 0.54926598072052
39 0.0
40 0.0
41 0.0
42 0.0
43 0.0
44 0.0
45 0.0
46 0.0
47 0.0
48 0.0
49 0.0
50 0.0
51 0.0
52 0.0
53 0.0
54 0.0


### 2.1.5. Save Word Embeddings Model

In [0]:
# Please comment your code

In [0]:
skip_gram_model.save("skip_gram_model.model")

In [0]:
#upload to Google Drive

In [0]:
uploaded_skip_gram_model_File = drive.CreateFile()

uploaded_skip_gram_model_File.SetContentFile('skip_gram_model.model')
uploaded_skip_gram_model_File.Upload()
print('uploaded_skip_gram_model_File with ID {}'.format(uploaded_skip_gram_model_File.get('id')))

uploaded_skip_gram_model_File with ID 1Uz4UG3VdXI0Swpk394eimD9Puo5JkfLf


### 2.1.6. Load Word Embeddings Model

In [0]:
# Please comment your code

In [0]:
from gensim.models import Word2Vec

/usr/local/lib/python3.6/dist-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


In [0]:
# Download from Google Drive
#1Uz4UG3VdXI0Swpk394eimD9Puo5JkfLf


downloaded_skip_gram_model_File = drive.CreateFile({'id': '1Uz4UG3VdXI0Swpk394eimD9Puo5JkfLf'}) #checked
downloaded_skip_gram_model_File.GetContentFile('skip_gram_model.model')  

In [0]:
#load downlaoded word2vec skip gram model
restored_skip_gram_model = Word2Vec.load("skip_gram_model.model")

## 2.2. Seq2Seq model

### 2.2.1. Apply/Import Word Embedding Model

####Facebook Pretrain Fasttext 'wiki.simple.bin

In [0]:
downloaded = drive.CreateFile({'id':'1fbDly6Tt8bkVV8OUTeCkz9UxQERfemn7'}) #checked
downloaded.GetContentFile('wiki.simple.bin') 

In [0]:
from gensim.models import FastText



In [0]:
fasttext_model = FastText.load_fasttext_format('wiki.simple.bin') #globally used

In [0]:
fasttext_model.wv.most_similar('man')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('woman', 0.576684296131134),
 ('manú', 0.504209578037262),
 ('womanhood', 0.48218244314193726),
 ('madwoman', 0.46441566944122314),
 ('maná', 0.4639086127281189),
 ('womanly', 0.46173131465911865),
 ('englishwoman', 0.4577977657318115),
 ('caïman', 0.45079147815704346),
 ('spiderman', 0.4464653730392456),
 ('spider', 0.44589245319366455)]

### 2.2.2. Build Seq2Seq Model

*You are required to describe how hyperparameters were decided with justification of your decision.*

I mainly focus on tuning learning rate, changing number hidden units, and #iterations. 

I found that models with proper learning rate will converge to low training loss such as below 0.5, with inappropreiate learning rate such as ... professional model would stuck on training loss of 1-2 and never fall down.

I also experiences 128, 256 number of hidden layer, in the case of professional model , 256 may lower down the training loss in less iteration but at the same time more training time and it seems does not have much differnt interms of chatbot chatting ability after all. So I stick with 256.

The most cretical hyperparameter is learning rate as mention a inappropraite learning rate could cause model stuck in a training loss that never fall down regardless of increasing number of iterations.

From experience, I tried range 0.000001 - 0.01, and I found for all 3 models(professional, friend and comic), 0.0005-0.001 worked for the best for all 3 models. The models converge to low traning loss ~=0.3 in 3000-5000 iterations. The model is also robust in Prediction stage in chatbot in temrs of performance of how chatbot answering user's questions, it gives reasonable responses and with similar input the chatbot gives same output.

I also tried multiple attemptions for the same set of hyper parameters values and each time the training loss could end up quite differnce, probably due to the random initialisation so I chose the one which converge better.

#### Make Batch for training and prediction

In [0]:
## Generate batch

In [0]:
'''
helper methods
'''
# get token index vector of questions and add paddings if the word is shorter than the maximum number of words
def get_vectors_q(sentence, max_input_words_amount):
    
    # tokenise the sentence
    tokenized_sentence = word_tokenize(sentence)
    tokenized_sentence = [token for token in tokenized_sentence if token.isalpha()] #remove puntuation, to lower case
    tokenized_sentence = [token.lower() for token in tokenized_sentence] #testing: puntuation, to lower case
  
    
    diff = max_input_words_amount - len(tokenized_sentence)
    
    # add paddings if the word is shorter than the maximum number of words    
    for x in range(diff):
        tokenized_sentence.append('_P_')          
        
    #data = tokens_to_ids(tokenized_sentence)
    data = tokens_to_wv(tokenized_sentence) #testing
    
        
    return data

# get token index vector of answer
def get_vectors_a(sentence, num_dic):    
    tokenized_sentence = [sentence]
    data = tokens_to_ids(tokenized_sentence, num_dic)
    
    return data
    

# convert tokens to index
def tokens_to_ids(tokenized_sentence, num_dic):
    ids = []

    for token in tokenized_sentence:
        if token in num_dic:
            ids.append(num_dic[token])
        else:
            ids.append(num_dic['_U_'])    

    return ids
  
#convert tokens to word vector 
def tokens_to_wv(tokenized_sentence):
    wvs = []

    for token in tokenized_sentence:
        if token in fasttext_model.wv:#testing
          wvs.append(fasttext_model.wv.__getitem__(token))
        else:
          wvs.append(fasttext_model.wv.__getitem__('_U_')) #TODO: testing, come back later    

    return wvs


'''
MAIN METHOD
'''
# generate a batch data for training/testing
def make_batch(seq_data, max_input_words_amount, num_dic, dic_len):
    input_batch = []
    output_batch = []
    target_batch = []

    for seq in seq_data:
        # Input for encoder cell, convert question to vector
        input_data = get_vectors_q(seq[0], max_input_words_amount) #get_vectors_q
           
        
        # Input for decoder cell, Add '_B_' at the beginning of the sequence data
        output_data = [num_dic['_B_']]
        output_data += get_vectors_a(seq[1], num_dic) #get_vectors_a
        
        # Output of decoder cell (Actual result), Add '_E_' at the end of the sequence data
        target = get_vectors_a(seq[1], num_dic) #get_vectors_a
        target.append(num_dic['_E_'])
        
        # Convert each token vector to one-hot encode data
        #input_batch.append(np.eye(dic_len)[input_data])
        input_batch.append(input_data)#testing
        output_batch.append(np.eye(dic_len)[output_data])
        
        target_batch.append(target)

    return input_batch, output_batch, target_batch




In [0]:
## TODO:test make batch

####FastText pretrained model vocab size

In [0]:
dim_fasttext = 300

#### Prepare Graph  for Professional Mode

In [0]:
### Setting Hyperparameters
professional_learning_rate = 0.001
professional_n_hidden = 128

professional_n_class = professional_dic_len
professional_n_input = professional_dic_len


professional_g = tf.Graph()
with professional_g.as_default():
  # encoder/decoder shape = [batch size, time steps, input size]
  professional_enc_input = tf.placeholder(tf.float32, [None, None, dim_fasttext], name='professional_enc_input') #testing
  professional_dec_input = tf.placeholder(tf.float32, [None, None, professional_n_input], name='professional_dec_input')

  # target shape = [batch size, time steps]
  professional_targets = tf.placeholder(tf.int64, [None, None], name='professional_targets')


  # Encoder Cell
  with tf.variable_scope('encode'):
      professional_enc_cell = tf.nn.rnn_cell.BasicRNNCell(professional_n_hidden)
      professional_enc_cell = tf.nn.rnn_cell.DropoutWrapper(professional_enc_cell, output_keep_prob=0.5)

      professional_outputs, professional_enc_states = tf.nn.dynamic_rnn(professional_enc_cell, professional_enc_input,
                                              dtype=tf.float32)
  # Decoder Cell
  with tf.variable_scope('decode'):
      professional_dec_cell = tf.nn.rnn_cell.BasicRNNCell(professional_n_hidden)
      professional_dec_cell = tf.nn.rnn_cell.DropoutWrapper(professional_dec_cell, output_keep_prob=0.5)

      # [IMPORTANT] Setting enc_states as inital_state of decoder cell
      professional_outputs, professional_dec_states = tf.nn.dynamic_rnn(professional_dec_cell, professional_dec_input,
                                              initial_state=professional_enc_states,
                                              dtype=tf.float32)

  professional_model = tf.layers.dense(professional_outputs, professional_n_class, activation=None, name='professional_model')

  professional_cost = tf.reduce_mean(
              tf.nn.sparse_softmax_cross_entropy_with_logits(
                  logits=professional_model, labels=professional_targets))

  professional_optimizer = tf.train.AdamOptimizer(professional_learning_rate).minimize(professional_cost)

#### #### Prepare Graph  for Friend Mode

In [0]:
### Setting Hyperparameters
friend_learning_rate = 0.001 
friend_n_hidden = 128
friend_n_class = friend_dic_len
friend_n_input = friend_dic_len


friend_g = tf.Graph()
with friend_g.as_default():
  # encoder/decoder shape = [batch size, time steps, input size]
  friend_enc_input = tf.placeholder(tf.float32, [None, None, dim_fasttext], name='friend_enc_input') #testing
  friend_dec_input = tf.placeholder(tf.float32, [None, None, friend_n_input], name='friend_dec_input')

  # target shape = [batch size, time steps]
  friend_targets = tf.placeholder(tf.int64, [None, None], name='friend_targets')


  # Encoder Cell
  with tf.variable_scope('encode'):
      friend_enc_cell = tf.nn.rnn_cell.BasicRNNCell(friend_n_hidden)
      friend_enc_cell = tf.nn.rnn_cell.DropoutWrapper(friend_enc_cell, output_keep_prob=0.5)

      friend_outputs, friend_enc_states = tf.nn.dynamic_rnn(friend_enc_cell, friend_enc_input,
                                              dtype=tf.float32)
  # Decoder Cell
  with tf.variable_scope('decode'):
      friend_dec_cell = tf.nn.rnn_cell.BasicRNNCell(friend_n_hidden)
      friend_dec_cell = tf.nn.rnn_cell.DropoutWrapper(friend_dec_cell, output_keep_prob=0.5)

      # [IMPORTANT] Setting enc_states as inital_state of decoder cell
      friend_outputs,friend_dec_states = tf.nn.dynamic_rnn(friend_dec_cell, friend_dec_input,
                                              initial_state=friend_enc_states,
                                              dtype=tf.float32)

  friend_model = tf.layers.dense(friend_outputs, friend_n_class, activation=None, name='friend_model')

  friend_cost = tf.reduce_mean(
              tf.nn.sparse_softmax_cross_entropy_with_logits(
                  logits=friend_model, labels=friend_targets))

  friend_optimizer = tf.train.AdamOptimizer(friend_learning_rate).minimize(friend_cost)

#### #### Prepare Graph  for Comic Mode

In [0]:
### Setting Hyperparameters
comic_learning_rate = 0.001
comic_n_hidden = 128

comic_n_class = comic_dic_len
comic_n_input = comic_dic_len

#dim_fasttext = 300

### Neural Network Model
#tf.reset_default_graph() #!!!!!!!!!!


comic_g = tf.Graph()
with comic_g.as_default():
  # encoder/decoder shape = [batch size, time steps, input size]
  comic_enc_input = tf.placeholder(tf.float32, [None, None, dim_fasttext], name='comic_enc_input') #testing
  comic_dec_input = tf.placeholder(tf.float32, [None, None, comic_n_input], name='comic_dec_input')

  # target shape = [batch size, time steps]
  comic_targets = tf.placeholder(tf.int64, [None, None], name='comic_targets')


  # Encoder Cell
  with tf.variable_scope('comic_encode'):
      comic_enc_cell = tf.nn.rnn_cell.BasicRNNCell(comic_n_hidden)
      #comic_enc_cell = tf.nn.rnn_cell.DropoutWrapper(comic_enc_cell, output_keep_prob=0.5)

      comic_outputs, comic_enc_states = tf.nn.dynamic_rnn(comic_enc_cell, comic_enc_input,
                                              dtype=tf.float32)
  # Decoder Cell
  with tf.variable_scope('comic_decode'):
      comic_dec_cell = tf.nn.rnn_cell.BasicRNNCell(comic_n_hidden)
      #comic_dec_cell = tf.nn.rnn_cell.DropoutWrapper(comic_dec_cell, output_keep_prob=0.5)

      # [IMPORTANT] Setting enc_states as inital_state of decoder cell
      comic_outputs, comic_dec_states = tf.nn.dynamic_rnn(comic_dec_cell, comic_dec_input,
                                              initial_state=comic_enc_states,
                                              dtype=tf.float32)

  comic_model = tf.layers.dense(comic_outputs, comic_n_class, activation=None, name='comic_model')

  comic_cost = tf.reduce_mean(
              tf.nn.sparse_softmax_cross_entropy_with_logits(
                  logits=comic_model, labels=comic_targets))

  comic_optimizer = tf.train.AdamOptimizer(comic_learning_rate).minimize(comic_cost)

### 2.2.3. Train Seq2Seq Model

In [0]:
#TODO: call  get_unique_word(file_name)

####Train Model in Professional Mode

In [0]:
with tf.Session(graph=professional_g) as professional_sess:
  professional_saver = tf.train.Saver() #testing
  ### Training Model
  #sess = tf.Session()
  professional_sess.run(tf.global_variables_initializer())
  
  # Generate a batch data
  professional_input_batch, professional_output_batch, professional_target_batch = make_batch(professional_seq_data, professional_max_input_words_amount, professional_num_dic, professional_dic_len)
  #print(professional_input_batch)
  
  professional_total_epoch = 5000

  for epoch in range(professional_total_epoch):
      _, loss = professional_sess.run([professional_optimizer, professional_cost],
                         feed_dict={professional_enc_input: professional_input_batch,
                                    professional_dec_input: professional_output_batch,
                                    professional_targets: professional_target_batch})
      if epoch % 100 == 0:
          print('Epoch:', '%04d' % (epoch + 1),
                'cost =', '{:.6f}'.format(loss))

  print('Epoch:', '%04d' % (epoch + 1),
        'cost =', '{:.6f}'.format(loss))
  print('Training completed')
  
  # Save the model - This will generate files: 
  # 1) .index, 2) .meta, 3) model_final.data-#-of-# (.data-00000-of-00001) 4)checkpoint
  save_path_professional_model = professional_saver.save(professional_sess, 'professional_model')
  
  #make sure it is proper graph and session
  print(professional_sess)
  print(professional_sess.graph)
  
  

Epoch: 0001 cost = 6.395020
Epoch: 0101 cost = 2.288866
Epoch: 0201 cost = 2.268923
Epoch: 0301 cost = 2.251377
Epoch: 0401 cost = 2.252035
Epoch: 0501 cost = 2.244484
Epoch: 0601 cost = 2.181035
Epoch: 0701 cost = 1.967153
Epoch: 0801 cost = 1.838742
Epoch: 0901 cost = 1.719705
Epoch: 1001 cost = 1.662776
Epoch: 1101 cost = 1.577468
Epoch: 1201 cost = 1.499170
Epoch: 1301 cost = 1.444769
Epoch: 1401 cost = 1.391792
Epoch: 1501 cost = 1.339973
Epoch: 1601 cost = 1.302789
Epoch: 1701 cost = 1.267084
Epoch: 1801 cost = 1.132770
Epoch: 1901 cost = 1.160611
Epoch: 2001 cost = 1.004207
Epoch: 2101 cost = 0.919288
Epoch: 2201 cost = 0.893837
Epoch: 2301 cost = 0.835093
Epoch: 2401 cost = 0.792085
Epoch: 2501 cost = 0.741288
Epoch: 2601 cost = 0.703055
Epoch: 2701 cost = 0.687220
Epoch: 2801 cost = 0.650854
Epoch: 2901 cost = 0.884748
Epoch: 3001 cost = 0.592742
Epoch: 3101 cost = 0.586575
Epoch: 3201 cost = 0.536444
Epoch: 3301 cost = 0.638616
Epoch: 3401 cost = 0.534779
Epoch: 3501 cost = 0

In [0]:
#restore previuos model for further training

####Train Model in Friend Mode


In [0]:
with tf.Session(graph=friend_g) as friend_sess:
  ### Training Model
  friend_saver = tf.train.Saver()#testing
  friend_sess.run(tf.global_variables_initializer())
  
  # Generate a batch data
  friend_input_batch, friend_output_batch, friend_target_batch = make_batch(friend_seq_data, friend_max_input_words_amount, friend_num_dic, friend_dic_len)

  friend_total_epoch = 5000

  for epoch in range(friend_total_epoch):
      _, loss = friend_sess.run([friend_optimizer, friend_cost],
                         feed_dict={friend_enc_input: friend_input_batch,
                                    friend_dec_input: friend_output_batch,
                                    friend_targets: friend_target_batch})
      if epoch % 100 == 0:
          print('Epoch:', '%04d' % (epoch + 1),
                'cost =', '{:.6f}'.format(loss))

  print('Epoch:', '%04d' % (epoch + 1),
        'cost =', '{:.6f}'.format(loss))
  print('Training completed')
  
  # Save the model - This will generate files: 
  # 1) .index, 2) .meta, 3) model_final.data-#-of-# (.data-00000-of-00001) 4)checkpoint
  save_path_friend_model = friend_saver.save(friend_sess, 'friend_model') #test folder
  print(save_path_friend_model)
  
  #make sure it is proper graph and session
  print(friend_sess)
  print(friend_sess.graph)
  

Epoch: 0001 cost = 6.473325
Epoch: 0101 cost = 2.308047
Epoch: 0201 cost = 2.271944
Epoch: 0301 cost = 2.259138
Epoch: 0401 cost = 2.235726
Epoch: 0501 cost = 2.149704
Epoch: 0601 cost = 1.898799
Epoch: 0701 cost = 1.784934
Epoch: 0801 cost = 1.516351
Epoch: 0901 cost = 1.648768
Epoch: 1001 cost = 1.307227
Epoch: 1101 cost = 1.193705
Epoch: 1201 cost = 1.142875
Epoch: 1301 cost = 1.016125
Epoch: 1401 cost = 0.990040
Epoch: 1501 cost = 1.051274
Epoch: 1601 cost = 0.834964
Epoch: 1701 cost = 0.835788
Epoch: 1801 cost = 0.782303
Epoch: 1901 cost = 0.735607
Epoch: 2001 cost = 0.717908
Epoch: 2101 cost = 0.723583
Epoch: 2201 cost = 0.661277
Epoch: 2301 cost = 0.816667
Epoch: 2401 cost = 0.602697
Epoch: 2501 cost = 0.597723
Epoch: 2601 cost = 0.744660
Epoch: 2701 cost = 0.524448
Epoch: 2801 cost = 0.502626
Epoch: 2901 cost = 0.466726
Epoch: 3001 cost = 0.448041
Epoch: 3101 cost = 0.988258
Epoch: 3201 cost = 0.410910
Epoch: 3301 cost = 0.447041
Epoch: 3401 cost = 0.374326
Epoch: 3501 cost = 0

In [0]:
'''testing'''
#fnie tuning



####Train Model in Comic Mode



In [0]:
# Add ops to save and restore all the variables.|
#comic_saver = tf.train.Saver()

In [0]:
with tf.Session(graph=comic_g) as comic_sess:
  ### Training Model
  #sess = tf.Session()
  
  comic_saver = tf.train.Saver() #testing
  comic_sess.run(tf.global_variables_initializer())
  
  # Generate a batch data
  comic_input_batch, comic_output_batch, comic_target_batch = make_batch(comic_seq_data, comic_max_input_words_amount, comic_num_dic, comic_dic_len)

  comic_total_epoch = 5000

  for epoch in range(comic_total_epoch):
      _, loss = comic_sess.run([comic_optimizer, comic_cost],
                         feed_dict={comic_enc_input: comic_input_batch,
                                    comic_dec_input: comic_output_batch,
                                    comic_targets: comic_target_batch})
      if epoch % 100 == 0:
          print('Epoch:', '%04d' % (epoch + 1),
                'cost =', '{:.6f}'.format(loss))

  print('Epoch:', '%04d' % (epoch + 1),
        'cost =', '{:.6f}'.format(loss))
  print('Training completed')
  
  # Save the model - This will generate files: 
  # 1) .index, 2) .meta, 3) model_final.data-#-of-# (.data-00000-of-00001) 4)checkpoint
  save_path_comic_model = comic_saver.save(comic_sess, 'comic_model')
  print(save_path_comic_model)
  
  #make sure it is proper graph and session
  print(comic_sess)
  print(comic_sess.graph)
  

Epoch: 0001 cost = 6.506395
Epoch: 0101 cost = 2.263892
Epoch: 0201 cost = 2.243112
Epoch: 0301 cost = 2.236284
Epoch: 0401 cost = 2.231743
Epoch: 0501 cost = 2.225033
Epoch: 0601 cost = 2.083419
Epoch: 0701 cost = 1.891839
Epoch: 0801 cost = 1.694991
Epoch: 0901 cost = 1.572294
Epoch: 1001 cost = 1.509540
Epoch: 1101 cost = 1.396104
Epoch: 1201 cost = 1.343648
Epoch: 1301 cost = 1.268636
Epoch: 1401 cost = 1.236469
Epoch: 1501 cost = 1.173554
Epoch: 1601 cost = 1.151036
Epoch: 1701 cost = 1.102404
Epoch: 1801 cost = 1.072992
Epoch: 1901 cost = 1.048710
Epoch: 2001 cost = 1.214587
Epoch: 2101 cost = 0.948927
Epoch: 2201 cost = 0.950745
Epoch: 2301 cost = 0.886970
Epoch: 2401 cost = 0.921890
Epoch: 2501 cost = 0.831043
Epoch: 2601 cost = 0.799232
Epoch: 2701 cost = 0.796359
Epoch: 2801 cost = 0.737574
Epoch: 2901 cost = 0.734993
Epoch: 3001 cost = 0.878240
Epoch: 3101 cost = 0.630640
Epoch: 3201 cost = 0.595913
Epoch: 3301 cost = 0.556427
Epoch: 3401 cost = 0.563885
Epoch: 3501 cost = 0

### 2.2.4. Save Seq2Seq Model

In [0]:
# function for saveing models 

In [0]:
def save_to_Google_Drive(save_path=''):
  ids_dic = {}
  #meta
  model_meta = drive.CreateFile()

  model_meta.SetContentFile(save_path+'.meta')
  model_meta.Upload()
  ids_dic['model_meta'] = model_meta.get('id')
  
  
  #index
  model_index = drive.CreateFile()

  model_index.SetContentFile(save_path+'.index')
  model_index.Upload()
  ids_dic['model_index'] = model_index.get('id')

  #data
  model_data = drive.CreateFile()

  model_data.SetContentFile(save_path+'.data-00000-of-00001')
  model_data.Upload()
  ids_dic['model_data'] = model_data.get('id')

  return ids_dic #ids

####Save Professional Model

In [0]:
# upload to google drive

In [0]:
professional_model_ids_dic = save_to_Google_Drive(save_path=save_path_professional_model)

In [0]:
print(professional_model_ids_dic)

{'model_meta': '1hU_f0WygZUNAazmf7l7HlgydKmJzppqU', 'model_index': '1e1HfWgHFLbSWfB02PFuuSEH9TuCMYvUV', 'model_data': '1nUtibZWTWt7f-Q9RjwRTgeXVzqy9Y2tk'}


In [0]:
print(save_path_professional_model)

./professional_model


####Save Friend Model

In [0]:
#upload .meta, .index .data
friend_model_ids_dic = save_to_Google_Drive(save_path=save_path_friend_model)


In [0]:
print(friend_model_ids_dic)

{'model_meta': '18497n1LxvZvoIE6SkcQGHMThQE5IMdNN', 'model_index': '1fuftYhI07uSzJWtipviGcC1DhYmnAxC7', 'model_data': '1DO2Rrsy8Dt8xjguG7d4utwHhknHgzz0l'}


####SaveTrained Comic Model

In [0]:
comic_model_ids_dic = save_to_Google_Drive(save_path=save_path_comic_model)

In [0]:
print(comic_model_ids_dic)

{'model_meta': '1Frc6MNssXyyRCASLFgIC8nnWkMA8az4A', 'model_index': '1pUnphD8D4lEw3nFC4CgLKC04y0TtJbPU', 'model_data': '1SuWS6zxK15xFne3p_z-VFlBKg7Pb_tXv'}


### 2.2.5. Load Seq2Seq Model

In [0]:
# Please comment your code

#### Download Model from Google Drive

#####Download Professional Model

In [0]:

##{'model_meta': '1DTDAC4X_o0mqdElezcifuasARe1v0fDM', 'model_index': '1vAlq_8k6GRjgC9Vg5fl2Ss9UC6jrW9GT', 'model_data': '1i1OmUpaH6A-1h4KhBSGSs0hqCXsIznP1'}

#{'model_meta': '1hU_f0WygZUNAazmf7l7HlgydKmJzppqU', 'model_index': '1e1HfWgHFLbSWfB02PFuuSEH9TuCMYvUV', 'model_data': '1nUtibZWTWt7f-Q9RjwRTgeXVzqy9Y2tk'}

In [0]:
#meta
downloaded_professional_model_meta = drive.CreateFile({'id': '1hU_f0WygZUNAazmf7l7HlgydKmJzppqU'})#checked
downloaded_professional_model_meta.GetContentFile('professional_model.meta')  

#index
downloaded_professional_model_data = drive.CreateFile({'id': '1e1HfWgHFLbSWfB02PFuuSEH9TuCMYvUV'})#checked
downloaded_professional_model_data.GetContentFile('professional_model.index')  

#data
downloaded_professional_model_data = drive.CreateFile({'id': '1nUtibZWTWt7f-Q9RjwRTgeXVzqy9Y2tk'})#checked
downloaded_professional_model_data.GetContentFile('professional_model.data-00000-of-00001')  


##### Download Friend Model

In [0]:

##{'model_meta': '1tbDS02nqSH69RyGqI28OyBQBuwqQvp9_', 'model_index': '1Zus7WZr9LpgTVl79XLT3drCFfKEYAXfy', 'model_data': '1v4VFddkyap9jPIw_p7bqAknjdxYXsve8'}

#{'model_meta': '18497n1LxvZvoIE6SkcQGHMThQE5IMdNN', 'model_index': '1fuftYhI07uSzJWtipviGcC1DhYmnAxC7', 'model_data': '1DO2Rrsy8Dt8xjguG7d4utwHhknHgzz0l'}

In [0]:
#meta
downloaded_friend_model_meta = drive.CreateFile({'id': '18497n1LxvZvoIE6SkcQGHMThQE5IMdNN'}) #checked
downloaded_friend_model_meta.GetContentFile('friend_model.meta')  

#index
downloaded_friend_model_index = drive.CreateFile({'id': '1fuftYhI07uSzJWtipviGcC1DhYmnAxC7'}) #checked
downloaded_friend_model_index.GetContentFile('friend_model.index')  

#data
downloaded_friend_model_data = drive.CreateFile({'id': '1DO2Rrsy8Dt8xjguG7d4utwHhknHgzz0l'}) #checked
downloaded_friend_model_data.GetContentFile('friend_model.data-00000-of-00001')  



##### Download Comic Model

In [0]:


##{'model_meta': '1aoiUiNDpx0YYZ8rQNBU6wNH9-bMC-JHU', 'model_index': '1v7ItFpzI_adN1sGcfbKen8ohUWteD0LF', 'model_data': '1BbdZWIyfDNhdmgEAfyM2G8AQyzYdJQzf'}

#{'model_meta': '1Frc6MNssXyyRCASLFgIC8nnWkMA8az4A', 'model_index': '1pUnphD8D4lEw3nFC4CgLKC04y0TtJbPU', 'model_data': '1SuWS6zxK15xFne3p_z-VFlBKg7Pb_tXv'}

In [0]:
#meta
downloaded_comic_model_meta = drive.CreateFile({'id': '1Frc6MNssXyyRCASLFgIC8nnWkMA8az4A'}) #checked
downloaded_comic_model_meta.GetContentFile('comic_model.meta')  

#index
downloaded_comic_model_index = drive.CreateFile({'id': '1pUnphD8D4lEw3nFC4CgLKC04y0TtJbPU'})#checked
downloaded_comic_model_index.GetContentFile('comic_model.index')  

#data
downloaded_comic_model_data = drive.CreateFile({'id': '1SuWS6zxK15xFne3p_z-VFlBKg7Pb_tXv'}) #checked
downloaded_comic_model_data.GetContentFile('comic_model.data-00000-of-00001')  

#### Load and Restore the model

In [0]:
#load and restore the model

####Retored graph and saved variables

#####Restore Professional Graphs, learned Weights

In [0]:
#PROFESSIONAL GRAPHA AND STORE VARIABLES

restored_professional_graph = tf.Graph()
with restored_professional_graph.as_default():
  restored_professional_graph_sess = tf.Session(graph=restored_professional_graph)
  restored_professional_saver = tf.train.import_meta_graph('./professional_model.meta')
  restored_professional_saver.restore(restored_professional_graph_sess, './professional_model')
  #create placeholders variables and
  professional_enc_input = restored_professional_graph.get_tensor_by_name("professional_enc_input:0")
  professional_dec_input = restored_professional_graph.get_tensor_by_name("professional_dec_input:0")

  professional_targets = restored_professional_graph.get_tensor_by_name("professional_targets:0")


  #Now, access the op that you want to run. 
  model_professional = restored_professional_graph.get_tensor_by_name("professional_model/BiasAdd:0")
  
  


Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./professional_model


##### Restore Friend Graphs, learned Weights

In [0]:
#FRIEND GRAPH AND STORE VARIABLES
restored_friend_graph = tf.Graph()
with restored_friend_graph.as_default():
  restored_friend_graph_sess = tf.Session(graph=restored_friend_graph)
  restored_friend_graph_saver = tf.train.import_meta_graph('./friend_model.meta')
  restored_friend_graph_saver.restore(restored_friend_graph_sess, './friend_model')
#create placeholders variables and
  friend_enc_input = restored_friend_graph.get_tensor_by_name("friend_enc_input:0")
  friend_dec_input = restored_friend_graph.get_tensor_by_name("friend_dec_input:0")

  friend_targets = restored_friend_graph.get_tensor_by_name("friend_targets:0")

  #Now, access the op that you want to run. 
  model_friend = restored_friend_graph.get_tensor_by_name("friend_model/BiasAdd:0")

INFO:tensorflow:Restoring parameters from ./friend_model


##### Restore Comic Graphs, learned Weights


In [0]:
#FRIEND GRAPH AND STORE VARIABLES
restored_comic_graph = tf.Graph()
with restored_comic_graph.as_default():
  restored_comic_graph_sess = tf.Session(graph=restored_comic_graph)
  restored_comic_graph_saver = tf.train.import_meta_graph('./comic_model.meta')
  restored_comic_graph_saver.restore(restored_comic_graph_sess, './comic_model')
#create placeholders variables and
  comic_enc_input = restored_comic_graph.get_tensor_by_name("comic_enc_input:0")
  comic_dec_input = restored_comic_graph.get_tensor_by_name("comic_dec_input:0")

  comic_targets = restored_comic_graph.get_tensor_by_name("comic_targets:0")

  #Now, access the op that you want to run. 
  model_comic = restored_comic_graph.get_tensor_by_name("comic_model/BiasAdd:0")

INFO:tensorflow:Restoring parameters from ./comic_model


# 3 - Evaluation (Running chatbot)

## 3.1. Start chatting

In [0]:
# Please comment your code

In [0]:
#return answer for specific personality
def get_answer(question_sentence, model, sess, enc_input, dec_input, targets, max_input_words_amount, unique_words, num_dic, dic_len):
  #set up batches for prediction
  max_output_words_amount=1

  seq_data = [question_sentence, '_U_' * max_output_words_amount]
  input_batch, output_batch, target_batch = make_batch([seq_data], max_input_words_amount, num_dic, dic_len)


  #prediction
  prediction = tf.argmax(model, 2)

  result = sess.run(prediction,
                    feed_dict={enc_input: input_batch,
                                    dec_input: output_batch,
                                    targets: target_batch})

  #Preprocessing answer
  # convert index number to actual token 
  decoded = [unique_words[i] for i in result[0]]

  # Remove anything after '_E_'        
  if "_E_" in decoded:
      end = decoded.index('_E_')
      translated = ' '.join(decoded[:end])
  else :
      translated = ' '.join(decoded[:])


  return translated
  

## 3.2. Change Personality

*Explain how to change personality (What is the command for changing personality?). *

**Instructions**



1.  First ask chatbot this trigger phrase: **change personality**
2.  Then user will be ask by chatbot: **what personality?**, then user shoudl response either **professional**, **friend** or **comic**





In [0]:
# Please comment your code

In [0]:
#start chat 
def run_chatbot():
  text_file = open("chatlog.txt", "w")
  
  
  #default professional 
  sess = restored_professional_graph_sess
  model= model_professional

  enc_input, dec_input, targets = professional_enc_input, professional_dec_input, professional_targets

  max_input_words_amount = professional_max_input_words_amount
  unique_words = professional_unique_words
  num_dic = professional_num_dic
  dic_len = professional_dic_len

  #flag for change personality
  change_personality = False


  print('Chatbot: Hi, human. Let\'s talk.')
  w = input('You: ')
  text_file.write('Chatbot: Hi, human. Let\'s talk.\n')
  text_file.write('You: {}\n'.format(w))
  #TODO: preproccesing on w???

  #keep alphabet only
  #all lower case


  while w:

    if w=='change personality':
      change_personality = True

      #response: what personality
      print('Chatbot: what personality? ')
      w = input('You: ')
      text_file.write('Chatbot: what personality?\n')
      text_file.write('You: {}\n'.format(w))
      #check if w is personality, of now ask same question again.
      while not (w=='professional' or w=='friend' or w=='comic'):
        print('Chatbot: what personality? ')
        w = input('You: ')
        text_file.write('Chatbot: what personality?\n')
        text_file.write('You: {}\n'.format(w))

      continue
    elif w=='professional' and change_personality:
      sess = restored_professional_graph_sess
      model= model_professional

      enc_input, dec_input, targets = professional_enc_input, professional_dec_input, professional_targets

      max_input_words_amount = professional_max_input_words_amount
      unique_words = professional_unique_words
      num_dic = professional_num_dic
      dic_len = professional_dic_len

      change_personality = False
      print('Chatbot: I have changed personality to professional. Let\'s chat.')
      w = input('You: ')
      text_file.write('Chatbot: I have changed personality to professional. Let\'s chat.\n')
      text_file.write('You: {}\n'.format(w))

    elif w=='friend' and change_personality:
      sess = restored_friend_graph_sess
      model= model_friend

      enc_input, dec_input, targets = friend_enc_input, friend_dec_input, friend_targets

      max_input_words_amount = friend_max_input_words_amount
      unique_words = friend_unique_words
      num_dic = friend_num_dic
      dic_len = friend_dic_len

      change_personality = False
      print('Chatbot: I have changed personality to friend. Let\'s chat.')
      w = input('You: ')
      text_file.write('Chatbot: I have changed personality to friend. Let\'s chat.\n')
      text_file.write('You: {}\n'.format(w))
      
    elif w=='comic' and change_personality:
      sess = restored_comic_graph_sess
      model= model_comic

      enc_input, dec_input, targets = comic_enc_input, comic_dec_input, comic_targets


      max_input_words_amount = comic_max_input_words_amount
      unique_words = comic_unique_words
      num_dic = comic_num_dic
      dic_len = comic_dic_len

      change_personality = False
      print('Chatbot: I have changed personality to comic. Let\'s chat.')
      w = input('You: ')
      text_file.write('Chatbot: I have changed personality to comic. Let\'s chat.\n')
      text_file.write('You: {}\n'.format(w))
      
      
    elif w=='bye bye chatbot.':
      break
  


    #get response from model
    answer = get_answer(w, model, sess, enc_input, dec_input, targets, max_input_words_amount, unique_words, num_dic, dic_len)
    print('Chatbot: {}'.format(answer))
    w = input('You: ')
    text_file.write('Chatbot: {}\n'.format(answer))
    text_file.write('You: {}\n'.format(w))

  
  print('Chatbot: Thank you. See you later.')
  text_file.write('Chatbot: Thank you. See you later.\n')
  text_file.close()

## 3.3. Save chat log

In [0]:
# Please comment your code

###Run chatbot


In [0]:
run_chatbot()

Chatbot: Hi, human. Let's talk.
You: are you dumb?
Chatbot: I try, but I don't always get it right.
You: That made no sense
Chatbot: I think I might have gotten lost there.
You: Will you be my best friend?
Chatbot: Certainly.
You: I want to spend the rest of my life with you
Chatbot: I think it's best if we stick to a professional relationship.
You:  want to spend the rest of my life with you
Chatbot: Certainly.
You: I think it's best if we stick to a professional relationship.
Chatbot: It's nice to meet you as well.
You: I have 7 cats
Chatbot: I'm flattered.
You: I am tall
Chatbot: Ok.
You: I am tall
Chatbot: Ok.
You: change personality
Chatbot: what personality? 
You: hihi
Chatbot: what personality? 
You: no
Chatbot: what personality? 
You: change personality
Chatbot: what personality? 
You: comci
Chatbot: what personality? 
You: comic
Chatbot: I have changed personality to comic. Let's chat.
You: I feel sad
Chatbot: Sorry to hear that. Here's a virtual high five if that will help.
Y

### Upload chatbot to google drive

In [0]:
#upload saved chatlog.txt to google drive
uploaded_chatlog = drive.CreateFile()
uploaded_chatlog.SetContentFile('chatlog.txt')
uploaded_chatlog.Upload()

print('uploaded_chatlog with ID {}'.format(uploaded_chatlog.get('id'))) #checked

uploaded_chatlog with ID 1Pq9UvMDXaKH7z9zvrcEX5Ti1mcm6kPXy


## 3.4. End chatting

**Instruction**

To end conversation, say **bye bye chatbot.** (be aware there is a . at the end of bye bye chatbot) to the chatbot then chatbot will response: **Thank you. See you later.** and quit


In [0]:
# Please comment your code

In [0]:
# bye bye chatbot.

## 3.5. Execute program

***Please make sure your program  is running properly.***

***Functions for downloading (from Google Drive) and loading models (both word embeddings and Seq2Seq) need to be called!*** 


### 3.5.1. Execute program - training mode

*Please include lines to train the bot.*

**INSTRUCTIONS**

Steps to download and load models of Section** 2.1-Word Embedding**


1.  run all cells from **1.1. Download Dataset (Personality chat datasets)** to set up pydrive
2. run all cells from  **2.1.6. Load Word Embeddings Model** to download and load trainded skip gram model



**INSTRUCTIONS**

Steps to download and load models of Section** 2.2-Seq2Seq. Model**



1.  Run all cells from **1-Data Preprocessing(Personality chat dataset)**
  *   Run all from **1.1. Download Dataset (Personality chat datasets)** to set up pydrive for later uses and download dataset
  *  Run all  from  **1.2 Proprocess data(Personality chat datasets)** to preprocess data set for future uses of make_batch


2.   Run all cells from **2.2.1 Apply/Import Word Embedding Model**  to import pretrained fasttext for seq2seq input
3. **2.2.2 Build Seq2Seq Model**
  *  need run only cells from TITLE **Make Batch for training and prediction**
  * (if not work please run all the cells on the  whole 2.2.2  Build Seq2Seq Mode)


4. Run all cells from **2.2.5 Load Seq2Seq Model** to download models from Google Drive and restored graphs




### 3.5.2. Execute program - chatting mode

*Please include lines to start chatting with the bot.*

**INSTRUCTIONS**


2.  Before start chatbot need to make sure already followed INSTRUCTIONS from section **3.5.1 execute program - training mode** and run everything as to load saved model 
3.  Run everything from **3.1 Start Chatting** for creating restored Graoph that download from google drive, then run everything from ** 3.2 Change Personality**  to define helper function required to run chatbot : get_answer() and run_chat()
4. Run the below cell: **run_chatbot()**


In [0]:
run_chatbot()

In [0]:
# Please comment your code


## Object Oriented Programming codes here

*If you have multiple classes use multiple code snippets to add them.*

In [0]:
# If you used OOP style, use this sectioon